In [1]:
import pandas as pd
import statsmodels.api as sm

data = pd.read_csv('C:/Users/inho0/Downloads/confer/data_set/final.csv')

# 독립 변수와 종속 변수 설정
X = data[['bus_station_cnt', 'traffic_cnt', 'pleasure_cnt', 'seoul_police_cnt', 'park_cnt', 'crossroad_cnt',
          '도로10이하_cnt', '도로10_20_cnt', '도로20_30_cnt', '도로30이상_cnt', 'child_cnt', 'silver_cnt', 'crosswalk_cnt']]
y = data['caslt_cnt']

# 독립 변수에 상수항 추가
X = sm.add_constant(X)

# 포아송 회귀 모델 적용
poisson_model = sm.GLM(y, X, family=sm.families.Poisson()).fit()

# 모델 요약 출력
print(poisson_model.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:              caslt_cnt   No. Observations:                 1200
Model:                            GLM   Df Residuals:                     1186
Model Family:                 Poisson   Df Model:                           13
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2831.1
Date:                Fri, 05 Jul 2024   Deviance:                       4960.9
Time:                        23:00:34   Pearson chi2:                 1.40e+04
No. Iterations:                     8   Pseudo R-squ. (CS):             0.6542
Covariance Type:            nonrobust                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -1.5146      0.091  

In [4]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# 음이항 회귀 모델 적용
negbin_model = smf.glm(formula='caslt_cnt ~ bus_station_cnt + traffic_cnt + pleasure_cnt + seoul_police_cnt + park_cnt + crossroad_cnt + 도로10이하_cnt + 도로10_20_cnt + 도로20_30_cnt + 도로30이상_cnt + child_cnt + silver_cnt + crosswalk_cnt', 
                       data=data, 
                       family=sm.families.NegativeBinomial()).fit()

# 모델 요약 출력
print(negbin_model.summary())


c:\Users\inho0\Downloads\confer\tobigs\lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                 Generalized Linear Model Regression Results                  
Dep. Variable:              caslt_cnt   No. Observations:                 1200
Model:                            GLM   Df Residuals:                     1186
Model Family:        NegativeBinomial   Df Model:                           13
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1530.4
Date:                Fri, 05 Jul 2024   Deviance:                       1954.9
Time:                        20:53:40   Pearson chi2:                 6.42e+03
No. Iterations:                    12   Pseudo R-squ. (CS):             0.4031
Covariance Type:            nonrobust                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -2.0501      0.146  

In [ ]:
# 유의미한 변수:

# bus_station_cnt: 버스 정류장 수는 두 모델 모두에서 매우 유의미하며, 사고 건수를 증가시키는 요인으로 작용
# traffic_cnt: 신호등 수 역시 두 모델 모두에서 매우 유의미하며, 사고 건수를 증가시키는 요
# pleasure_cnt: 유흥업소 수는 두 모델에서 모두 유의미하게 사고 건수를 증가시키는 요인
# seoul_police_cnt: 경찰서 수는 두 모델에서 모두 유의미하게 사고 건수를 감소시키는 요인
# park_cnt: 공원 수는 두 모델에서 모두 유의미하게 사고 건수를 감소시키는 요인
# 도로10이하_cnt: 도로폭 10m 이하 도로 수는 두 모델에서 모두 유의미하게 사고 건수를 증가시키는 요인
# 도로20_30_cnt: 도로폭 20-30m 도로 수는 두 모델에서 모두 유의미하게 사고 건수를 크게 감소시키는 요인
# silver_cnt: 노인 보호 구역 수는 두 모델에서 모두 유의미하게 사고 건수를 증가시키는 요인

# 유의미하지 않은 변수: crossroad_cnt, 도로10_20_cnt, 도로30이상_cnt, child_cnt, crosswalk_cnt 변수는 두 모델 모두에서 유의미하지 않음
